<a href="https://colab.research.google.com/github/smkalle/ai_projects/blob/main/tutorials/microgpt/microgpt_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MicroGPT: Building a GPT from Scratch in Pure Python

> *"The most atomic way to train and inference a GPT in pure, dependency-free Python.*
> *This file is the complete algorithm. Everything else is just efficiency."*
> -- **Andrej Karpathy**

## The Core Insight

A GPT language model -- the same architecture behind ChatGPT, Claude, and every modern LLM --
can be reduced to **six atomic math operations** on individual scalar numbers:

| Operation | Code | Derivative | Purpose |
|-----------|------|-----------|---------|
| Addition | `a + b` | `da=1, db=1` | Combine values |
| Multiplication | `a * b` | `da=b, db=a` | Scale values |
| Power | `a ** n` | `da = n * a^(n-1)` | Non-linear transform |
| Logarithm | `log(a)` | `da = 1/a` | Compress range (loss) |
| Exponential | `exp(a)` | `da = exp(a)` | Expand range (softmax) |
| ReLU | `max(0, a)` | `da = 1 if a>0 else 0` | Activation gate |

**That's it.** The entire architecture -- embeddings, attention, MLP, normalization,
loss function -- is built from just these six operations on scalars. A tiny autograd
engine (30 lines) tracks every operation and computes gradients via the chain rule.
Adam optimizer updates the parameters. The model learns.

**No PyTorch. No TensorFlow. No NumPy. Just Python + these 6 operations.**

In [1]:
import os
import math
import random
from datetime import datetime

random.seed(42)

print("Imports: os, math, random, datetime")
print("No PyTorch, TensorFlow, NumPy, or any other library needed.")

Imports: os, math, random, datetime
No PyTorch, TensorFlow, NumPy, or any other library needed.


---
## Part 1: The Autograd Engine

The `Value` class wraps a single number and tracks the computation graph.
When you do `a + b`, it records that `a` and `b` were the inputs, and stores the
derivative rule so gradients can be computed later by `backward()`.

Each of the 6 operations stores its **local derivative** (from calculus):
- `d(a+b)/da = 1` -- adding doesn't change the rate
- `d(a*b)/da = b` -- scaling by b
- `d(a^n)/da = n * a^(n-1)` -- power rule
- `d(log a)/da = 1/a`
- `d(exp a)/da = exp(a)` -- the exponential is its own derivative
- `d(relu a)/da = 1 if a > 0 else 0` -- pass-through or block

In [2]:
class Value:
    """A scalar value with automatic differentiation.

    This is the ENTIRE autograd engine. Every number in the neural network
    is a Value. Math on Values builds a computation graph. Then .backward()
    walks that graph in reverse to compute gradients via the chain rule.
    """
    __slots__ = ('data', 'grad', '_children', '_local_grads')

    def __init__(self, data, children=(), local_grads=()):
        self.data = data
        self.grad = 0
        self._children = children
        self._local_grads = local_grads

    def __repr__(self):
        return f"Value({self.data:.6f})"

    # --- THE 6 ATOMIC OPERATIONS ---

    def __add__(self, other):            # a + b    -> da=1, db=1
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data + other.data, (self, other), (1, 1))

    def __mul__(self, other):            # a * b    -> da=b, db=a
        other = other if isinstance(other, Value) else Value(other)
        return Value(self.data * other.data, (self, other), (other.data, self.data))

    def __pow__(self, other):            # a ** n   -> da = n * a^(n-1)
        return Value(self.data**other, (self,), (other * self.data**(other-1),))

    def log(self):                       # log(a)   -> da = 1/a
        return Value(math.log(self.data), (self,), (1/self.data,))

    def exp(self):                       # exp(a)   -> da = exp(a)
        return Value(math.exp(self.data), (self,), (math.exp(self.data),))

    def relu(self):                      # max(0,a) -> da = 1 if a>0 else 0
        return Value(max(0, self.data), (self,), (float(self.data > 0),))

    # --- DERIVED OPERATIONS (built from the 6 above) ---
    def __neg__(self): return self * -1
    def __radd__(self, other): return self + other
    def __sub__(self, other): return self + (-other)
    def __rsub__(self, other): return other + (-self)
    def __rmul__(self, other): return self * other
    def __truediv__(self, other): return self * other**-1
    def __rtruediv__(self, other): return other * self**-1

    def backward(self):
        """Backpropagate gradients through the entire computation graph."""
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._children:
                    build_topo(child)
                topo.append(v)
        build_topo(self)
        self.grad = 1
        for v in reversed(topo):
            for child, local_grad in zip(v._children, v._local_grads):
                child.grad += local_grad * v.grad
        return len(topo)

print("Value class defined -- the complete autograd engine")
print("6 atomic ops: +, *, **, log, exp, relu")

Value class defined -- the complete autograd engine
6 atomic ops: +, *, **, log, exp, relu


### Test: Forward Computation
Verify all 6 operations compute correct values.

In [3]:
print("=== Testing the 6 Atomic Operations ===\n")

a, b = Value(3.0), Value(4.0)
tests = [
    ("add",    (a + b).data,             3.0 + 4.0),
    ("mul",    (a * b).data,             3.0 * 4.0),
    ("pow",    (a ** 2).data,            3.0 ** 2),
    ("log",    Value(2.0).log().data,    math.log(2.0)),
    ("exp",    Value(1.0).exp().data,    math.exp(1.0)),
    ("relu+",  Value(5.0).relu().data,   5.0),
    ("relu-",  Value(-3.0).relu().data,  0.0),
]
all_pass = True
for name, got, expected in tests:
    ok = abs(got - expected) < 1e-10
    all_pass = all_pass and ok
    print(f"  {name:6s}: got {got:10.6f}  expected {expected:10.6f}  [{'PASS' if ok else 'FAIL'}]")

print("\n=== Derived Operations ===\n")
x = Value(6.0)
for name, got, expected in [("neg", (-x).data, -6.0), ("sub", (Value(10.0)-x).data, 4.0), ("div", (x/Value(3.0)).data, 2.0)]:
    ok = abs(got - expected) < 1e-10
    all_pass = all_pass and ok
    print(f"  {name:6s}: got {got:10.6f}  expected {expected:10.6f}  [{'PASS' if ok else 'FAIL'}]")

assert all_pass
print("\nAll forward tests passed.")

=== Testing the 6 Atomic Operations ===

  add   : got   7.000000  expected   7.000000  [PASS]
  mul   : got  12.000000  expected  12.000000  [PASS]
  pow   : got   9.000000  expected   9.000000  [PASS]
  log   : got   0.693147  expected   0.693147  [PASS]
  exp   : got   2.718282  expected   2.718282  [PASS]
  relu+ : got   5.000000  expected   5.000000  [PASS]
  relu- : got   0.000000  expected   0.000000  [PASS]

=== Derived Operations ===

  neg   : got  -6.000000  expected  -6.000000  [PASS]
  sub   : got   4.000000  expected   4.000000  [PASS]
  div   : got   2.000000  expected   2.000000  [PASS]

All forward tests passed.


### Test: Gradient Computation (Backward Pass)

**Exercise**: For each test, try to compute the gradient BY HAND before reading the answer.
Remember the chain rule: if `f = g(h(x))`, then `df/dx = dg/dh * dh/dx`

In [4]:
print("=== Gradient Tests ===")
print("Predict each gradient BEFORE reading the output!\n")

# Test 1: f = x + y
x, y = Value(3.0), Value(4.0); f = x + y; f.backward()
print(f"Test 1: f = x + y       df/dx = {x.grad} (rule: 1)  df/dy = {y.grad} (rule: 1)")
assert abs(x.grad - 1.0) < 1e-10

# Test 2: f = x * y
x, y = Value(3.0), Value(4.0); f = x * y; f.backward()
print(f"Test 2: f = x * y       df/dx = {x.grad} (rule: b=4)  df/dy = {y.grad} (rule: a=3)")
assert abs(x.grad - 4.0) < 1e-10

# Test 3: f = x^3
x = Value(2.0); f = x ** 3; f.backward()
print(f"Test 3: f = x^3         df/dx = {x.grad} (power rule: 3*2^2=12)")
assert abs(x.grad - 12.0) < 1e-10

# Test 4: CHAIN RULE
x = Value(3.0); f = (x * 2 + 1) ** 2; f.backward()
print(f"Test 4: f = (2x+1)^2    df/dx = {x.grad} (chain: 2*(2*3+1)*2 = 28)")
assert abs(x.grad - 28.0) < 1e-10

# Test 5: DEEP CHAIN
x = Value(1.0); f = (x.exp() + 1).log(); f.backward()
expected = math.exp(1) / (math.exp(1) + 1)
print(f"Test 5: f = log(exp(x)+1)  df/dx = {x.grad:.6f} (sigmoid = {expected:.6f})")
assert abs(x.grad - expected) < 1e-6

print("\nAll gradient tests passed!")

=== Gradient Tests ===
Predict each gradient BEFORE reading the output!

Test 1: f = x + y       df/dx = 1 (rule: 1)  df/dy = 1 (rule: 1)
Test 2: f = x * y       df/dx = 4.0 (rule: b=4)  df/dy = 3.0 (rule: a=3)
Test 3: f = x^3         df/dx = 12.0 (power rule: 3*2^2=12)
Test 4: f = (2x+1)^2    df/dx = 28.0 (chain: 2*(2*3+1)*2 = 28)
Test 5: f = log(exp(x)+1)  df/dx = 0.731059 (sigmoid = 0.731059)

All gradient tests passed!


### Verifying Autograd: Numerical Gradient Checking

We verify autograd against **numerical differentiation** (brute-force ground truth):
```
df/dx ~ (f(x+h) - f(x-h)) / (2h)    where h ~ 1e-5
```
Slow but reliable. If both match, our engine is correct.

In [5]:
def check_gradients(build_fn, inputs, names=None, h=1e-5):
    """Verify autograd vs numerical finite differences."""
    if names is None: names = [f"x{i}" for i in range(len(inputs))]
    vals = [Value(x) for x in inputs]
    output = build_fn(vals); output.backward()
    auto_grads = [v.grad for v in vals]
    num_grads = []
    for i in range(len(inputs)):
        vp = [Value(x) for x in inputs]; vp[i].data += h
        vm = [Value(x) for x in inputs]; vm[i].data -= h
        num_grads.append((build_fn(vp).data - build_fn(vm).data) / (2*h))
    print(f"  {'Input':>8s}  {'Autograd':>12s}  {'Numerical':>12s}  {'Match':>6s}")
    print(f"  {'-'*44}")
    ok = True
    for nm, ag, ng in zip(names, auto_grads, num_grads):
        m = abs(ag - ng) < h * 100; ok = ok and m
        print(f"  {nm:>8s}  {ag:12.6f}  {ng:12.6f}  {'OK' if m else 'FAIL':>6s}")
    return ok

print("=== Numerical Gradient Verification ===\n")
print("Test 1: f(x,y) = x^2 * y + y^3")
ok1 = check_gradients(lambda v: v[0]**2 * v[1] + v[1]**3, [3.0, 2.0], ['x','y'])
print("\nTest 2: f(x) = relu(log(exp(x)+1) * x^2)  [all 6 ops]")
ok2 = check_gradients(lambda v: ((v[0].exp()+1).log() * v[0]**2).relu(), [1.5], ['x'])
print("\nTest 3: cross-entropy loss (the EXACT loss we train with)")
def xent(v):
    mx = max(vi.data for vi in v)
    exps = [(vi - mx).exp() for vi in v]
    return -(exps[0] / sum(exps)).log()
ok3 = check_gradients(xent, [1.0, 2.0, 3.0], ['a','b','c'])
assert ok1 and ok2 and ok3
print("\nAll numerical checks passed -- autograd is correct!")

=== Numerical Gradient Verification ===

Test 1: f(x,y) = x^2 * y + y^3
     Input      Autograd     Numerical   Match
  --------------------------------------------
         x     12.000000     12.000000      OK
         y     21.000000     21.000000      OK

Test 2: f(x) = relu(log(exp(x)+1) * x^2)  [all 6 ops]
     Input      Autograd     Numerical   Match
  --------------------------------------------
         x      6.943782      6.943782      OK

Test 3: cross-entropy loss (the EXACT loss we train with)
     Input      Autograd     Numerical   Match
  --------------------------------------------
         a     -0.909969     -0.909969      OK
         b      0.244728      0.244728      OK
         c      0.665241      0.665241      OK

All numerical checks passed -- autograd is correct!


### Counting the Computation Graph
Every operation creates a node. Let's count them to understand scale.

In [6]:
def count_graph_nodes(v):
    """Count nodes in the computation graph."""
    visited = set(); ops = 0; leaves = 0
    def walk(node):
        nonlocal ops, leaves
        if node not in visited:
            visited.add(node)
            if node._children:
                ops += 1
                for c in node._children: walk(c)
            else: leaves += 1
    walk(v)
    return {'total': len(visited), 'ops': ops, 'leaves': leaves}

x, y = Value(3.0), Value(4.0)
f = (x + y) * (x - y)
s = count_graph_nodes(f)
print(f"f = (x+y)*(x-y) = {f.data}")
print(f"  Graph: {s['total']} nodes = {s['ops']} ops + {s['leaves']} leaves")

x = Value(2.0); f = (x**2 + x*3 + 1).log()
s = count_graph_nodes(f)
print(f"\nf = log(x^2 + 3x + 1) = {f.data:.6f}")
print(f"  Graph: {s['total']} nodes = {s['ops']} ops + {s['leaves']} leaves")
print(f"\nEvery node has a tracked derivative. backward() walks them all.")

f = (x+y)*(x-y) = -7.0
  Graph: 7 nodes = 4 ops + 3 leaves

f = log(x^2 + 3x + 1) = 2.397895
  Graph: 8 nodes = 5 ops + 3 leaves

Every node has a tracked derivative. backward() walks them all.


---
## Part 2: Dataset & Tokenization

We train on ~32K human names. **Character-level tokenization**: each unique character
gets an ID (`a`=0, `b`=1, ..., `z`=25), plus a **BOS** (Beginning of Sequence) token.

In [7]:
if not os.path.exists('input.txt'):
    print("Downloading names dataset...")
    import urllib.request
    url = 'https://raw.githubusercontent.com/karpathy/makemore/refs/heads/master/names.txt'
    urllib.request.urlretrieve(url, 'input.txt')
docs = [l.strip() for l in open('input.txt').read().strip().split('\n') if l.strip()]
random.shuffle(docs)

uchars = sorted(set(''.join(docs)))
BOS = len(uchars)
vocab_size = len(uchars) + 1

def encode(text): return [uchars.index(ch) for ch in text]
def decode(tokens): return ''.join(uchars[t] if t < len(uchars) else '<BOS>' for t in tokens)

print(f"Dataset: {len(docs):,} names")
print(f"Vocabulary: {vocab_size} tokens = {len(uchars)} chars + BOS(={BOS})")
print(f"Sample: {docs[:8]}")
for name in ["alice", "bob", "zara"]:
    assert decode(encode(name)) == name
print("\nEncode/decode roundtrip verified.")

Dataset: 32,033 names
Vocabulary: 27 tokens = 26 chars + BOS(=26)
Sample: ['yuheng', 'diondre', 'xavien', 'jori', 'juanluis', 'erandi', 'phia', 'samatha']

Encode/decode roundtrip verified.


### Training Sequence Format
For `emma`: `[BOS, e, m, m, a, BOS]`. Model predicts each next character:
```
BOS->'e'  'e'->'m'  'm'->'m'  'm'->'a'  'a'->BOS
```

In [8]:
print("=== Training Sequence Examples ===\n")
for name in ["emma", "bob", "lily"]:
    tokens = [BOS] + encode(name) + [BOS]
    print(f"\'{name}\' -> {tokens}")
    for i in range(len(tokens)-1):
        ic = uchars[tokens[i]] if tokens[i] < len(uchars) else 'BOS'
        tc = uchars[tokens[i+1]] if tokens[i+1] < len(uchars) else 'BOS'
        print(f"    pos {i}: \'{ic}\' -> \'{tc}\'")
    print()

=== Training Sequence Examples ===

'emma' -> [26, 4, 12, 12, 0, 26]
    pos 0: 'BOS' -> 'e'
    pos 1: 'e' -> 'm'
    pos 2: 'm' -> 'm'
    pos 3: 'm' -> 'a'
    pos 4: 'a' -> 'BOS'

'bob' -> [26, 1, 14, 1, 26]
    pos 0: 'BOS' -> 'b'
    pos 1: 'b' -> 'o'
    pos 2: 'o' -> 'b'
    pos 3: 'b' -> 'BOS'

'lily' -> [26, 11, 8, 11, 24, 26]
    pos 0: 'BOS' -> 'l'
    pos 1: 'l' -> 'i'
    pos 2: 'i' -> 'l'
    pos 3: 'l' -> 'y'
    pos 4: 'y' -> 'BOS'



---
## Part 3: Building Blocks (All from Atomic Operations)

Every component is a composition of the 6 atomic ops. Autograd tracks every one.

### 3.1: Linear Layer
`output = W @ x` -- matrix-vector multiply. For 16x16: ~496 atomic ops.

In [9]:
def linear(x, w):
    """Matrix-vector multiply. Built from multiply + add."""
    return [sum(wi * xi for wi, xi in zip(wo, x)) for wo in w]

# Test
x_t = [Value(1.0), Value(2.0), Value(3.0)]
w_t = [[Value(1),Value(0),Value(0)], [Value(0),Value(1),Value(0)]]
out = linear(x_t, w_t)
assert out[0].data == 1.0 and out[1].data == 2.0
print("linear: identity test passed")

print("\nGradient check:")
def lin_fn(v):
    x = v[:3]; w = [[v[3],v[4],v[5]], [v[6],v[7],v[8]]]
    return sum(linear(x, w))
check_gradients(lin_fn, [1.,2.,3.,.5,.3,.1,.2,.4,.6],
                ['x0','x1','x2','w00','w01','w02','w10','w11','w12'])

x16 = [Value(0.)]*16; w16 = [[Value(0.)]*16 for _ in range(16)]
s = sum(linear(x16, w16)); st = count_graph_nodes(s)
print(f"\nlinear(16->16): {st['ops']} atomic ops")

linear: identity test passed

Gradient check:
     Input      Autograd     Numerical   Match
  --------------------------------------------
        x0      0.700000      0.700000      OK
        x1      0.700000      0.700000      OK
        x2      0.700000      0.700000      OK
       w00      1.000000      1.000000      OK
       w01      2.000000      2.000000      OK
       w02      3.000000      3.000000      OK
       w10      1.000000      1.000000      OK
       w11      2.000000      2.000000      OK
       w12      3.000000      3.000000      OK

linear(16->16): 528 atomic ops


### 3.2: Softmax
Converts logits to probabilities summing to 1. Uses: subtract, exp, add, divide.

In [10]:
def softmax(logits):
    """Logits -> probabilities (sum to 1)."""
    max_val = max(val.data for val in logits)
    exps = [(val - max_val).exp() for val in logits]
    total = sum(exps)
    return [e / total for e in exps]

probs = softmax([Value(1.0), Value(2.0), Value(3.0)])
psum = sum(p.data for p in probs)
assert abs(psum - 1.0) < 1e-6
print(f"softmax([1,2,3]) = [{', '.join(f'{p.data:.4f}' for p in probs)}]  sum={psum:.6f}")

print("\nGradient check (-log softmax = cross-entropy):")
check_gradients(lambda v: -(softmax(v)[0]).log(), [1.,2.,3.], ['a','b','c'])

softmax([1,2,3]) = [0.0900, 0.2447, 0.6652]  sum=1.000000

Gradient check (-log softmax = cross-entropy):
     Input      Autograd     Numerical   Match
  --------------------------------------------
         a     -0.909969     -0.909969      OK
         b      0.244728      0.244728      OK
         c      0.665241      0.665241      OK


True

### 3.3: RMSNorm
Normalize to consistent scale: `output = x / sqrt(mean(x^2) + eps)`

In [11]:
def rmsnorm(x):
    """Root Mean Square normalization."""
    ms = sum(xi * xi for xi in x) / len(x)
    scale = (ms + 1e-5) ** -0.5
    return [xi * scale for xi in x]

x_t = [Value(1.), Value(2.), Value(3.), Value(4.)]
x_n = rmsnorm(x_t)
ms = sum(v.data**2 for v in x_n) / len(x_n)
print(f"rmsnorm([1,2,3,4]) = [{', '.join(f'{v.data:.4f}' for v in x_n)}]")
print(f"  Mean of squares: {ms:.6f} (should be ~1.0)")

print("\nGradient check:")
check_gradients(lambda v: sum(rmsnorm(v)), [1.,2.,3.,4.], ['x0','x1','x2','x3'])

rmsnorm([1,2,3,4]) = [0.3651, 0.7303, 1.0954, 1.4606]
  Mean of squares: 0.999999 (should be ~1.0)

Gradient check:
     Input      Autograd     Numerical   Match
  --------------------------------------------
        x0      0.243432      0.243432      OK
        x1      0.121716      0.121716      OK
        x2      0.000000      0.000000      OK
        x3     -0.121715     -0.121715      OK


True

### Operation Count Summary

In [12]:
print("=== Atomic Operations per Component ===\n")
for label, fn in [
    ("linear(16->16)", lambda: sum(linear([Value(0.)]*16, [[Value(0.)]*16 for _ in range(16)]))),
    ("linear(16->64)", lambda: sum(linear([Value(0.)]*16, [[Value(0.)]*16 for _ in range(64)]))),
    ("softmax(27)",    lambda: sum(softmax([Value(.1*i) for i in range(27)]))),
    ("rmsnorm(16)",    lambda: sum(rmsnorm([Value(.1*i) for i in range(16)]))),
]:
    s = count_graph_nodes(fn())
    print(f"  {label:20s}: {s['ops']:5d} ops")
print(f"\nA full forward pass chains THOUSANDS of these together.")

=== Atomic Operations per Component ===

  linear(16->16)      :   528 ops
  linear(16->64)      :  2112 ops
  softmax(27)         :   162 ops
  rmsnorm(16)         :    67 ops

A full forward pass chains THOUSANDS of these together.


---
## Part 4: Model Parameters

~8,000 learnable Values. Config: 16-dim embedding, 4 heads, 1 layer, 16 context.

In [13]:
n_embd = 16; n_head = 4; n_layer = 1; block_size = 16
head_dim = n_embd // n_head

def matrix(nout, nin, std=0.08):
    return [[Value(random.gauss(0, std)) for _ in range(nin)] for _ in range(nout)]

state_dict = {'wte': matrix(vocab_size, n_embd), 'wpe': matrix(block_size, n_embd),
              'lm_head': matrix(vocab_size, n_embd)}
for i in range(n_layer):
    state_dict[f'layer{i}.attn_wq'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wk'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wv'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.attn_wo'] = matrix(n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc1'] = matrix(4*n_embd, n_embd)
    state_dict[f'layer{i}.mlp_fc2'] = matrix(n_embd, 4*n_embd)

params = [p for mat in state_dict.values() for row in mat for p in row]

print("=== Model Parameters ===\n")
total = 0
for name, mat in state_dict.items():
    r, c = len(mat), len(mat[0]); n = r*c; total += n
    print(f"  {name:25s}  {r:3d} x {c:3d} = {n:5d}")
print(f"  {'TOTAL':25s}           {total:5d}")

=== Model Parameters ===

  wte                         27 x  16 =   432
  wpe                         16 x  16 =   256
  lm_head                     27 x  16 =   432
  layer0.attn_wq              16 x  16 =   256
  layer0.attn_wk              16 x  16 =   256
  layer0.attn_wv              16 x  16 =   256
  layer0.attn_wo              16 x  16 =   256
  layer0.mlp_fc1              64 x  16 =  1024
  layer0.mlp_fc2              16 x  64 =  1024
  TOTAL                                4192


---
## Part 5: The GPT Architecture
```
Token -> [Embed + Position] -> RMSNorm
      -> [Multi-Head Attention] + Residual -> RMSNorm
      -> [MLP: Linear->ReLU->Linear] + Residual
      -> [Linear -> Logits]
```
**Attention**: Q*K^T/sqrt(d) -> softmax -> weighted V. 4 heads in parallel.
**Residual**: out = layer(x) + x. **KV Cache**: past keys/values saved.

In [14]:
def gpt(token_id, pos_id, keys, values):
    """GPT forward pass for one token. Every op is atomic, tracked by autograd."""
    tok_emb = state_dict['wte'][token_id]
    pos_emb = state_dict['wpe'][pos_id]
    x = [t + p for t, p in zip(tok_emb, pos_emb)]
    x = rmsnorm(x)
    for li in range(n_layer):
        # Attention
        x_res = x; x = rmsnorm(x)
        q = linear(x, state_dict[f'layer{li}.attn_wq'])
        k = linear(x, state_dict[f'layer{li}.attn_wk'])
        v = linear(x, state_dict[f'layer{li}.attn_wv'])
        keys[li].append(k); values[li].append(v)
        x_attn = []
        for h in range(n_head):
            hs = h * head_dim
            q_h = q[hs:hs+head_dim]
            k_h = [ki[hs:hs+head_dim] for ki in keys[li]]
            v_h = [vi[hs:hs+head_dim] for vi in values[li]]
            al = [sum(q_h[j]*k_h[t][j] for j in range(head_dim))/head_dim**0.5
                  for t in range(len(k_h))]
            aw = softmax(al)
            x_attn.extend([sum(aw[t]*v_h[t][j] for t in range(len(v_h)))
                           for j in range(head_dim)])
        x = linear(x_attn, state_dict[f'layer{li}.attn_wo'])
        x = [a+b for a, b in zip(x, x_res)]
        # MLP
        x_res = x; x = rmsnorm(x)
        x = linear(x, state_dict[f'layer{li}.mlp_fc1'])
        x = [xi.relu() for xi in x]
        x = linear(x, state_dict[f'layer{li}.mlp_fc2'])
        x = [a+b for a, b in zip(x, x_res)]
    return linear(x, state_dict['lm_head'])

print(f"GPT defined: {n_layer} layer, {n_head} heads, {n_embd}-dim")

GPT defined: 1 layer, 4 heads, 16-dim


### Test: Forward Pass & Computation Graph

In [15]:
print("=== Forward Pass Test ===\n")
keys_t, values_t = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
logits = gpt(BOS, 0, keys_t, values_t)
probs = softmax(logits)
assert len(logits) == vocab_size
assert abs(sum(p.data for p in probs) - 1.0) < 1e-6

top5 = sorted(range(len(probs)), key=lambda i: probs[i].data, reverse=True)[:5]
print("Top 5 predictions (untrained = random):")
for r, idx in enumerate(top5, 1):
    ch = uchars[idx] if idx < len(uchars) else 'BOS'
    print(f"  {r}. \'{ch}\' = {probs[idx].data:.4f}")

st = count_graph_nodes(-probs[0].log())
print(f"\nComputation graph (1 token): {st['total']:,} nodes ({st['ops']:,} ops)")

=== Forward Pass Test ===

Top 5 predictions (untrained = random):
  1. 'o' = 0.0617
  2. 's' = 0.0545
  3. 'c' = 0.0541
  4. 'z' = 0.0506
  5. 'k' = 0.0497

Computation graph (1 token): 11,223 nodes (7,446 ops)


### Full Sequence Graph Analysis

In [16]:
print("=== Processing \'emma\' token by token ===\n")
tokens = [BOS] + encode("emma") + [BOS]; n = len(tokens)-1
keys_a, values_a = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
losses_a = []
for pos_id in range(n):
    tid, tgt = tokens[pos_id], tokens[pos_id+1]
    logits = gpt(tid, pos_id, keys_a, values_a)
    probs = softmax(logits); lt = -probs[tgt].log(); losses_a.append(lt)
    ic = uchars[tid] if tid<len(uchars) else 'BOS'
    tc = uchars[tgt] if tgt<len(uchars) else 'BOS'
    st = count_graph_nodes(lt)
    print(f"  pos {pos_id}: \'{ic}\'->\'{tc}\'  loss={lt.data:.4f}  graph={st['total']:,} nodes")
total_loss = (1/n) * sum(losses_a)
st = count_graph_nodes(total_loss)
print(f"\nTotal loss: {total_loss.data:.4f}")
print(f"Total graph: {st['total']:,} nodes ({st['ops']:,} ops)")
nn = total_loss.backward()
print(f"Backward traversed {nn:,} nodes")
print(f"Params with gradients: {sum(1 for p in params if p.grad!=0)}/{len(params)}")
for p in params: p.grad = 0

=== Processing 'emma' token by token ===

  pos 0: 'BOS'->'e'  loss=3.1097  graph=11,223 nodes
  pos 1: 'e'->'m'  loss=4.2117  graph=12,535 nodes
  pos 2: 'm'->'m'  loss=3.6083  graph=13,847 nodes
  pos 3: 'm'->'a'  loss=3.5067  graph=15,143 nodes
  pos 4: 'a'->'BOS'  loss=3.0409  graph=16,455 nodes

Total loss: 3.4955
Total graph: 43,091 nodes (38,116 ops)
Backward traversed 43,091 nodes
Params with gradients: 3616/4192


---
## Part 6: Anatomy of a Training Step

Before training 1000 steps, let's dissect ONE step:
1. **Forward**: name -> predictions -> loss
2. **Backward**: gradients via chain rule through entire graph
3. **Adam update**: adjust each parameter
4. **Zero**: reset gradients

In [17]:
print("=== One Training Step (Detailed) ===\n")
doc = docs[0]
tokens = [BOS] + [uchars.index(ch) for ch in doc] + [BOS]
n = min(block_size, len(tokens)-1)
print(f"Name: \'{doc}\'  ({n} predictions)\n")

# Forward
print("FORWARD:")
ks, vs = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
losses = []
for pos_id in range(n):
    tid, tgt = tokens[pos_id], tokens[pos_id+1]
    logits = gpt(tid, pos_id, ks, vs); probs = softmax(logits)
    lt = -probs[tgt].log(); losses.append(lt)
    ic = uchars[tid] if tid<len(uchars) else 'BOS'
    tc = uchars[tgt] if tgt<len(uchars) else 'BOS'
    top3 = sorted(range(len(probs)), key=lambda i: probs[i].data, reverse=True)[:3]
    t3 = " ".join(f"\'{uchars[i] if i<len(uchars) else 'BOS'}\':{probs[i].data:.2f}" for i in top3)
    print(f"  \'{ic}\'->\'{tc}\' loss={lt.data:.3f}  top3=[{t3}]  {'HIT' if tgt in top3 else ''}")
loss = (1/n)*sum(losses)
print(f"  Avg loss: {loss.data:.4f}\n")

# Backward
print("BACKWARD:")
nn = loss.backward()
grads = [abs(p.grad) for p in params if p.grad!=0]
print(f"  {nn:,} nodes traversed")
print(f"  {len(grads)}/{len(params)} params got gradients")
if grads: print(f"  |grad| range: [{min(grads):.6f}, {max(grads):.6f}]\n")

# Adam demo
print("ADAM UPDATE (example: param[0]):")
p0 = params[0]; old = p0.data
mh = (0.15*p0.grad)/(1-0.85); vh = (0.01*p0.grad**2)/(1-0.99)
upd = 0.01*mh/(vh**0.5+1e-8)
print(f"  grad={p0.grad:.6f} -> update={upd:.6f}")
print(f"  {old:.6f} -> {old-upd:.6f}")
for p in params: p.grad = 0

=== One Training Step (Detailed) ===

Name: 'yuheng'  (7 predictions)

FORWARD:
  'BOS'->'y' loss=3.628  top3=['o':0.06 's':0.05 'c':0.05]  
  'y'->'u' loss=3.215  top3=['q':0.06 'd':0.05 'r':0.05]  
  'u'->'h' loss=3.561  top3=['q':0.07 'e':0.07 'g':0.06]  
  'h'->'e' loss=3.542  top3=['k':0.08 'l':0.06 'c':0.05]  
  'e'->'n' loss=3.074  top3=['z':0.07 'e':0.06 'b':0.05]  
  'n'->'g' loss=3.238  top3=['e':0.06 'q':0.06 'n':0.05]  
  'g'->'BOS' loss=3.304  top3=['z':0.06 't':0.05 'q':0.05]  
  Avg loss: 3.3660

BACKWARD:
  59,647 nodes traversed
  3728/4192 params got gradients
  |grad| range: [0.000001, 0.341059]

ADAM UPDATE (example: param[0]):
  grad=0.000000 -> update=0.000000
  -0.042732 -> -0.042732


---
## Part 7: Training

Adam optimizer with linear LR decay. 1000 steps. Watch the loss decrease.

In [18]:
learning_rate = 0.01; beta1, beta2, eps_adam = 0.85, 0.99, 1e-8
m = [0.0]*len(params); v = [0.0]*len(params)
num_steps = 1000; loss_history = []

print(f"Training {num_steps} steps on {len(docs):,} names...\n")
start_time = datetime.now()

for step in range(num_steps):
    doc = docs[step % len(docs)]
    tokens = [BOS] + [uchars.index(ch) for ch in doc] + [BOS]
    n = min(block_size, len(tokens)-1)
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    losses = []
    for pos_id in range(n):
        token_id, target_id = tokens[pos_id], tokens[pos_id+1]
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax(logits)
        losses.append(-probs[target_id].log())
    loss = (1/n)*sum(losses); loss_history.append(loss.data)
    loss.backward()
    lr_t = learning_rate * (1 - step/num_steps)
    for i, p in enumerate(params):
        m[i] = beta1*m[i] + (1-beta1)*p.grad
        v[i] = beta2*v[i] + (1-beta2)*p.grad**2
        m_hat = m[i]/(1-beta1**(step+1)); v_hat = v[i]/(1-beta2**(step+1))
        p.data -= lr_t * m_hat / (v_hat**0.5 + eps_adam)
        p.grad = 0
    if (step+1) % 100 == 0 or step == 0:
        elapsed = (datetime.now()-start_time).total_seconds()
        avg = sum(loss_history[max(0,step-99):step+1])/min(step+1,100)
        print(f"  step {step+1:4d}/{num_steps} | loss {loss.data:.4f} | avg {avg:.4f} | "
              f"lr {lr_t:.4f} | {(step+1)/elapsed:.1f} s/s | \'{doc}\'")

elapsed = (datetime.now()-start_time).total_seconds()
print(f"\nDone in {elapsed:.0f}s. Final loss: {loss_history[-1]:.4f}")

Training 1000 steps on 32,033 names...

  step    1/1000 | loss 3.3660 | avg 3.3660 | lr 0.0100 | 2.4 s/s | 'yuheng'
  step  100/1000 | loss 3.3669 | avg 2.7725 | lr 0.0090 | 2.7 s/s | 'fatumata'
  step  200/1000 | loss 2.3097 | avg 2.5346 | lr 0.0080 | 2.8 s/s | 'ameliarose'
  step  300/1000 | loss 2.3178 | avg 2.4640 | lr 0.0070 | 2.9 s/s | 'dmani'
  step  400/1000 | loss 2.3428 | avg 2.4443 | lr 0.0060 | 2.9 s/s | 'bryler'
  step  500/1000 | loss 2.0645 | avg 2.4640 | lr 0.0050 | 3.0 s/s | 'soraia'
  step  600/1000 | loss 2.4851 | avg 2.4243 | lr 0.0040 | 3.0 s/s | 'brayven'
  step  700/1000 | loss 2.3357 | avg 2.4133 | lr 0.0030 | 3.0 s/s | 'neely'
  step  800/1000 | loss 2.2632 | avg 2.3896 | lr 0.0020 | 3.0 s/s | 'ayzlin'
  step  900/1000 | loss 2.7785 | avg 2.3340 | lr 0.0010 | 3.0 s/s | 'boyce'
  step 1000/1000 | loss 2.6497 | avg 2.2761 | lr 0.0000 | 3.0 s/s | 'akio'

Done in 334s. Final loss: 2.6497


### Training Analysis

In [19]:
print("=== Training Analysis ===\n")
for s, e in [(0,100),(100,300),(300,500),(500,700),(700,900),(900,1000)]:
    avg = sum(loss_history[s:e])/(e-s)
    print(f"  Steps {s+1:4d}-{e:4d}: {avg:.4f}  {'#'*int(avg*10)}")
imp = (loss_history[0]-loss_history[-1])/loss_history[0]*100
ppl = math.exp(sum(loss_history[-100:])/100)
print(f"\nImprovement: {loss_history[0]:.4f} -> {loss_history[-1]:.4f} ({imp:.1f}%)")
print(f"Final perplexity: {ppl:.1f} (random={vocab_size}, lower=better)")
print(f"Model is {vocab_size/ppl:.1f}x better than random guessing")

=== Training Analysis ===

  Steps    1- 100: 2.7725  ###########################
  Steps  101- 300: 2.4993  ########################
  Steps  301- 500: 2.4542  ########################
  Steps  501- 700: 2.4188  ########################
  Steps  701- 900: 2.3618  #######################
  Steps  901-1000: 2.2761  ######################

Improvement: 3.3660 -> 2.6497 (21.3%)
Final perplexity: 9.7 (random=27, lower=better)
Model is 2.8x better than random guessing


---
## Part 8: Generation

Sample from trained model: BOS -> predict -> sample -> feed back -> repeat.
**Temperature**: low=conservative, high=creative.

In [20]:
def generate(temperature=0.5):
    """Generate a name by sampling from the trained model."""
    keys, values = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    token_id = BOS; chars = []
    for pos_id in range(block_size):
        logits = gpt(token_id, pos_id, keys, values)
        probs = softmax([l/temperature for l in logits])
        token_id = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if token_id == BOS: break
        chars.append(uchars[token_id])
    return ''.join(chars)

print("generate() defined")

generate() defined


In [21]:
print("=== Step-by-step Generation ===\n")
for si in range(3):
    kg, vg = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    tid = BOS; chars = []
    print(f"Sample {si+1}:")
    for pos_id in range(block_size):
        logits = gpt(tid, pos_id, kg, vg)
        probs = softmax([l/0.5 for l in logits])
        top3 = sorted(range(len(probs)), key=lambda i: probs[i].data, reverse=True)[:3]
        t3 = " ".join(f"\'{uchars[i] if i<len(uchars) else 'BOS'}\':{probs[i].data:.2f}" for i in top3)
        tid = random.choices(range(vocab_size), weights=[p.data for p in probs])[0]
        if tid == BOS:
            print(f"  pos {pos_id}: [{t3}] -> BOS (done)"); break
        chars.append(uchars[tid])
        print(f"  pos {pos_id}: [{t3}] -> \'{uchars[tid]}\'")
    print(f"  Result: \'{''.join(chars)}\'\n")

=== Step-by-step Generation ===

Sample 1:
  pos 0: ['a':0.31 'k':0.12 'j':0.10] -> 'k'
  pos 1: ['a':0.77 'e':0.14 'i':0.05] -> 'a'
  pos 2: ['r':0.37 'n':0.27 'l':0.07] -> 'm'
  pos 3: ['i':0.44 'a':0.31 'e':0.17] -> 'o'
  pos 4: ['n':0.83 'r':0.07 'l':0.03] -> 'n'
  pos 5: ['BOS':0.84 'a':0.08 'n':0.06] -> BOS (done)
  Result: 'kamon'

Sample 2:
  pos 0: ['a':0.31 'k':0.12 'j':0.10] -> 'a'
  pos 1: ['n':0.41 'l':0.23 'r':0.16] -> 'n'
  pos 2: ['a':0.57 'e':0.11 'n':0.08] -> 'n'
  pos 3: ['a':0.58 'n':0.09 'e':0.08] -> BOS (done)
  Result: 'ann'

Sample 3:
  pos 0: ['a':0.31 'k':0.12 'j':0.10] -> 'k'
  pos 1: ['a':0.77 'e':0.14 'i':0.05] -> 'a'
  pos 2: ['r':0.37 'n':0.27 'l':0.07] -> 'r'
  pos 3: ['i':0.46 'a':0.25 'e':0.15] -> 'a'
  pos 4: ['n':0.64 'BOS':0.17 'r':0.06] -> 'i'
  pos 5: ['BOS':0.45 'n':0.23 'a':0.16] -> BOS (done)
  Result: 'karai'



In [22]:
print("=== Temperature Comparison ===\n")
for temp in [0.3, 0.5, 0.8, 1.0, 1.5]:
    names = [generate(temp) for _ in range(10)]
    label = {0.3:"conservative",0.5:"balanced",0.8:"exploratory",1.0:"creative",1.5:"wild"}[temp]
    print(f"  temp={temp} ({label:12s}): {', '.join(names)}")

print(f"\n{'='*60}")
print(f"  20 Generated Names (temperature=0.5)")
print(f"{'='*60}\n")
gen = []
for i in range(20):
    name = generate(0.5); gen.append(name); print(f"  {i+1:2d}. {name}")
print(f"\nUnique: {len(set(gen))}/20  Avg len: {sum(len(n) for n in gen)/len(gen):.1f}")

=== Temperature Comparison ===

  temp=0.3 (conservative): anare, sanara, elina, saran, anan, areli, jana, saya, anar, jarie
  temp=0.5 (balanced    ): manna, erilina, anchio, erien, onana, alela, anton, kamin, reoron, jana
  temp=0.8 (exploratory ): saiann, sha, jaighn, kasann, acahah, ganiy, d, ahsso, iara, sakna
  temp=1.0 (creative    ): ketts, teron, wynya, sana, jaynann, jibyn, moree, amabem, irlil, enngyn
  temp=1.5 (wild        ): macesuy, mhimara, viienny, raujeo, mij, ricina, paaplr, fafelas, panaml, ri

  20 Generated Names (temperature=0.5)

   1. delen
   2. sari
   3. dana
   4. cajana
   5. melezan
   6. elana
   7. tarar
   8. jaran
   9. annar
  10. arana
  11. maban
  12. azin
  13. jaren
  14. jaiel
  15. aniah
  16. aler
  17. jona
  18. tazen
  19. jainen
  20. madann

Unique: 20/20  Avg len: 5.0


---
## Part 9: Analysis & Experiments

In [23]:
print("=== Model: Real Names vs Nonsense ===\n")
for name in ["emma","liam","olivia","noah","ava","zzzzz","qqqq","aeiou","xyzw"]:
    tokens = [BOS]+encode(name)+[BOS]; n = len(tokens)-1
    kt, vt = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
    total = 0.
    for pos_id in range(n):
        logits = gpt(tokens[pos_id], pos_id, kt, vt)
        probs = softmax(logits)
        total += -probs[tokens[pos_id+1]].log().data
    avg = total/n; ppl = math.exp(avg)
    print(f"  \'{name:8s}\': loss={avg:.3f}  ppl={ppl:6.1f}  {'#'*min(40,int(avg*8))}")
print(f"\nReal names have lower loss than nonsense.")

=== Model: Real Names vs Nonsense ===

  'emma    ': loss=2.726  ppl=  15.3  #####################
  'liam    ': loss=2.315  ppl=  10.1  ##################
  'olivia  ': loss=2.248  ppl=   9.5  #################
  'noah    ': loss=2.513  ppl=  12.3  ####################
  'ava     ': loss=2.582  ppl=  13.2  ####################
  'zzzzz   ': loss=4.173  ppl=  64.9  #################################
  'qqqq    ': loss=7.187  ppl=1322.3  ########################################
  'aeiou   ': loss=3.533  ppl=  34.2  ############################
  'xyzw    ': loss=4.324  ppl=  75.5  ##################################

Real names have lower loss than nonsense.


In [24]:
print("=== Attention Patterns (\'{}\'): ===\n".format("sarah"))
name = "sarah"; tokens_v = [BOS]+encode(name); chars_v = ['BOS']+list(name)
kv, vv = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
all_attn = []
for pos_id in range(len(tokens_v)):
    tid = tokens_v[pos_id]
    x = [t+p for t,p in zip(state_dict['wte'][tid], state_dict['wpe'][pos_id])]
    x = rmsnorm(x); li = 0; x = rmsnorm(x)
    q = linear(x, state_dict[f'layer{li}.attn_wq'])
    k = linear(x, state_dict[f'layer{li}.attn_wk'])
    kv[li].append(k); vv[li].append(linear(x, state_dict[f'layer{li}.attn_wv']))
    pa = []
    for h in range(n_head):
        hs = h*head_dim; q_h = q[hs:hs+head_dim]
        k_h = [ki[hs:hs+head_dim] for ki in kv[li]]
        al = [sum(q_h[j]*k_h[t][j] for j in range(head_dim))/head_dim**0.5
              for t in range(len(k_h))]
        pa.append([w.data for w in softmax(al)])
    all_attn.append(pa)
for h in range(n_head):
    print(f"Head {h}:")
    print("       "+"".join(f"{c:>7s}" for c in chars_v))
    for pi in range(len(tokens_v)):
        row = f"  {chars_v[pi]:>4s}: "
        for t in range(pi+1):
            w = all_attn[pi][h][t]
            if w > 0.3: row += f" [{w:.2f}]"
            elif w > 0.1: row += f"  {w:.2f} "
            else: row += f"  .    "
        print(row)
    print()

=== Attention Patterns ('sarah'): ===

Head 0:
           BOS      s      a      r      a      h
   BOS:  [1.00]
     s:  [0.54] [0.46]
     a:  [0.34] [0.35] [0.30]
     r:   0.21  [0.32]  0.27   0.21 
     a:   0.24   0.19   0.22   0.17   0.18 
     h:   0.18   0.17   0.16   0.15   0.17   0.17 

Head 1:
           BOS      s      a      r      a      h
   BOS:  [1.00]
     s:  [0.48] [0.52]
     a:   0.21  [0.58]  0.20 
     r:   0.22   0.23   0.25   0.29 
     a:   0.11   0.16   .     [0.32] [0.32]
     h:   0.24   0.14   0.15   0.13   0.17   0.16 

Head 2:
           BOS      s      a      r      a      h
   BOS:  [1.00]
     s:  [0.58] [0.42]
     a:   .     [0.90]  .    
     r:   .     [0.58]  .      0.27 
     a:   .      0.21   .     [0.58]  0.12 
     h:  [0.39]  .     [0.35]  .      0.14   .    

Head 3:
           BOS      s      a      r      a      h
   BOS:  [1.00]
     s:  [0.45] [0.55]
     a:  [0.30] [0.69]  .    
     r:   0.24   0.27   0.27   0.22 
     a:   0.14   

In [25]:
print("=== Gradient Flow by Layer ===\n")
doc = docs[0]
tokens = [BOS]+[uchars.index(ch) for ch in doc]+[BOS]; n = min(block_size, len(tokens)-1)
kg, vg = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
losses = []
for pos_id in range(n):
    logits = gpt(tokens[pos_id], pos_id, kg, vg)
    probs = softmax(logits); losses.append(-probs[tokens[pos_id+1]].log())
loss = (1/n)*sum(losses); loss.backward()
for name, mat in state_dict.items():
    g = [abs(p.grad) for row in mat for p in row]
    mg = sum(g)/len(g); mx = max(g)
    print(f"  {name:25s}: mean|g|={mg:.6f}  max={mx:.6f}  {'#'*min(30,int(mg*500))}")
for p in params: p.grad = 0
print(f"\nLarger gradients = more learning in that component")

=== Gradient Flow by Layer ===

  wte                      : mean|g|=0.016977  max=0.273229  ########
  wpe                      : mean|g|=0.028649  max=0.273229  ##############
  lm_head                  : mean|g|=0.042936  max=0.468020  #####################
  layer0.attn_wq           : mean|g|=0.002691  max=0.030330  #
  layer0.attn_wk           : mean|g|=0.002738  max=0.026094  #
  layer0.attn_wv           : mean|g|=0.014620  max=0.076216  #######
  layer0.attn_wo           : mean|g|=0.016016  max=0.077669  ########
  layer0.mlp_fc1           : mean|g|=0.009036  max=0.132025  ####
  layer0.mlp_fc2           : mean|g|=0.005368  max=0.082592  ##

Larger gradients = more learning in that component


---
## Part 10: Exercises

### Exercise 1: Predict the Gradient
Before running: `f(x) = x^2 + 3x + 1` at `x = 2`. What is `df/dx`?

In [26]:
x = Value(2.0); f = x**2 + x*3 + 1; f.backward()
print(f"f(2) = {f.data} (expected 11)  df/dx = {x.grad} (expected 7)")

f(2) = 11.0 (expected 11)  df/dx = 7.0 (expected 7)


### Exercise 2: Count the Math
How many operations for 'emma'? Estimate, then run:

In [27]:
tokens = [BOS]+encode("emma")+[BOS]; n = len(tokens)-1
kc, vc = [[] for _ in range(n_layer)], [[] for _ in range(n_layer)]
lc = []
for pos_id in range(n):
    logits = gpt(tokens[pos_id], pos_id, kc, vc)
    probs = softmax(logits); lc.append(-probs[tokens[pos_id+1]].log())
st = count_graph_nodes((1/n)*sum(lc))
print(f"'emma' ({n} predictions): {st['total']:,} graph nodes, {st['ops']:,} operations")
print(f"Over {num_steps} training steps: ~{st['ops']*num_steps:,} total ops")
print(f"All differentiated by our 30-line autograd engine!")

'emma' (5 predictions): 43,091 graph nodes, 38,116 operations
Over 1000 training steps: ~38,116,000 total ops
All differentiated by our 30-line autograd engine!


### Exercise 3: Experiment
Try changing hyperparameters (re-run from Part 4):
```python
n_embd = 32       # more capacity
n_layer = 2       # deeper
learning_rate = 0.1   # too high?
num_steps = 2000  # more training
temperature = 0.1 # very conservative
temperature = 2.0 # very random
```

---
## Summary

**The 6 atomic operations:** `+`, `*`, `**`, `log`, `exp`, `relu`

**Everything is composition:**
Linear = mul + add | Softmax = sub + exp + add + div | RMSNorm = mul + add + pow
Attention = linear + softmax + linear | MLP = linear + relu + linear | Loss = softmax + log

**What's different in production LLMs (GPT-4, Claude)?**
Same architecture, billions of parameters, GPU tensors instead of scalar Values,
CUDA kernels instead of Python loops. **But the math is identical.**

> *"This file is the complete algorithm. Everything else is just efficiency."*

**Next steps:**
- [Karpathy: Zero to Hero (YouTube)](https://www.youtube.com/playlist?list=PLAqhIrjkxbuWI23v9cThsA9GvCAUhRvKZ)
- [Attention Is All You Need](https://arxiv.org/abs/1706.03762)
- Implement in PyTorch for 100x speedup